In [155]:
import pandas as pd
import matplotlib.pyplot as plt
import os
import numpy as np
from tqdm import tqdm
%matplotlib inline

In [295]:
df = pd.read_csv('/mnt/c/Users/Derek/Desktop/mass-suite/example_data/batch/test1.txt', sep="\\t", engine='python')
df2 = pd.read_csv('/mnt/c/Users/Derek/Desktop/mass-suite/example_data/batch/test2.txt', sep="\\t", engine='python')
df3 = pd.read_csv('/mnt/c/Users/Derek/Desktop/mass-suite/example_data/batch/test3.txt', sep="\\t", engine='python')

In [296]:
df = df[['RT (min)','Precursor m/z','Area']]
df2 = df2[['RT (min)','Precursor m/z','Area']]
df3 = df3[['RT (min)','Precursor m/z','Area']]

In [253]:
df

,RT (min),Precursor m/z,Area
0,0.578250,179.0358,26772.347656
1,0.583783,201.0726,9090.625000
2,0.583783,217.0489,17694.775391
3,0.589317,142.0350,17614.511719
4,0.589317,158.0119,64892.605469
...,...,...,...
1178,21.163620,204.9978,17016.837891
1179,21.169150,167.9657,19431.066406
1180,21.185750,163.9683,39115.789062
1181,21.207900,158.1247,3277.271240


In [64]:
df
df.append(df2.iloc[3,:], ignore_index=True)
#df['Average RT (min)'] = df2.iloc[1,:]
#df.sort_values(by='RT (min)')
df.rename(columns={'RT (min)':'Average RT (min)'}, inplace=True)
df.at[1, 'Average RT (min)'] = (df.iloc[1, 0] + df2.iloc[1, 0])
df

,Average RT (min),Precursor m/z,Area
0,0.578250,179.0358,26772.347656
1,1.748117,201.0726,9090.625000
2,0.583783,217.0489,17694.775391
3,0.589317,142.0350,17614.511719
4,0.589317,158.0119,64892.605469
...,...,...,...
1178,21.163620,204.9978,17016.837891
1179,21.169150,167.9657,19431.066406
1180,21.185750,163.9683,39115.789062
1181,21.207900,158.1247,3277.271240


In [42]:
#df2.iloc[1,2]
#df2['blah']=0
#df2
RT_error = 0.05
mz_error = 0.00002*df.iloc[:, 1]
overlap=np.where(((df.iloc[:, 0] - RT_error) <= df2.iloc[1000, 0]) & (df2.iloc[1000, 0] <= (df.iloc[:,0] + RT_error))
       & ((df.iloc[:, 1] - mz_error) <= df2.iloc[1000, 1]) & (df2.iloc[1000, 1] <= (df.iloc[:, 1] + mz_error)))
overlap[0].size
#overlap_val=pd.DataFrame(a)
#overlap_val
#overlap_val.iloc[0,0]
#df.at[overlap_val.iloc[0,0], 'Area'] = df2.iloc[0,2]
#np.where(df2['RT (min)'] > 10)
#key = pd.DataFrame(a)
#key.iloc[0,0]

1

In [93]:
alignment_df=pd.DataFrame()
alignment_df=alignment_df.append(df.iloc[1, 0:2], ignore_index = True)
#alignment_df['Average RT (min)'] = df.iloc[:,0]
#alignment_df.rename(columns={'RT (min)':'Average RT (min)'}, inplace=True)
alignment_df

,Precursor m/z,RT (min)
0,201.0726,0.583783


In [85]:
#df2.at[key.iloc[0,0],'Area']=df.iloc[3,2]
df.append(df2.iloc[1, 0:2], ignore_index = True)

,RT (min),Precursor m/z,Area
0,0.578250,179.0358,26772.347656
1,0.583783,201.0726,9090.625000
2,0.583783,217.0489,17694.775391
3,0.589317,142.0350,17614.511719
4,0.589317,158.0119,64892.605469
...,...,...,...
1179,21.169150,167.9657,19431.066406
1180,21.185750,163.9683,39115.789062
1181,21.207900,158.1247,3277.271240
1182,21.307500,217.0474,11130.147461


plt.scatter(df['RT (min)'],df['Precursor m/z'])
plt.scatter(df2['RT (min)'],df2['Precursor m/z'])

In [303]:
RT_error = 0.05
mz_error = 0.00002*df.iloc[:,1]
overlaps=[]
count = 0
for row in range(len(df3)):
    overlap = np.where(((df.iloc[:, 0] - RT_error) <= df3.iloc[row, 0]) &\
                       (df3.iloc[row, 0] <= (df.iloc[:,0] + RT_error))\
                        & ((df.iloc[:, 1] - mz_error) <= df3.iloc[row, 1])\
                        & (df3.iloc[row, 1] <= (df.iloc[:, 1] + mz_error)))
    if overlap[0].size == 0:
        count += 1
    else:
        pass
    overlaps.append(overlap)

In [304]:
%time print(count)

125
CPU times: user 0 ns, sys: 0 ns, total: 0 ns
Wall time: 307 µs


In [41]:
overlap = np.where(((df.iloc[:, 0] - RT_error) <= df2.iloc[10, 0]) &\
                       (df2.iloc[10, 0] <= (df.iloc[:,0] + RT_error))\
                        & ((df.iloc[:, 1] - mz_error) <= df2.iloc[10, 1])\
                        & (df2.iloc[10, 1] <= (df.iloc[:, 1] + mz_error)))
overlap = np.array([0])
a=np.where(overlap<-1)
a[0].size

0

In [290]:
def realignment(batch_path, batch_name):
    """This function works by using one .txt file as a reference in which 
    other files realigned to in terms of precursor and RT."""
    RT_error = 0.10 # units of minutes, can be adjusted
    alignment_df = pd.DataFrame()
    # defines number of files in batch
    num_files = len([f for f in os.listdir(batch_path)if os.path.isfile(os.path.join(batch_path, f))])
    for files in tqdm(range(num_files)):
        standard_sample = os.listdir(batch_path)[0] # first sample
        # reads .txt file into a dataframe
        ref_df = pd.read_csv(batch_path + standard_sample, sep="\\t", engine='python')
        standard_df = ref_df[['RT (min)','Precursor m/z', 'Area']] 
        if files > 0: # alignment only starts past first file
            align_sample = os.listdir(batch_path)[files]
            df = pd.read_csv(batch_path + align_sample, sep="\\t", engine='python')
            sample_df = df[['RT (min)','Precursor m/z', 'Area']]
            # inserts a blank column with header as file name
            alignment_df[align_sample] = 0
            for row in range(len(sample_df)):
                overlap = np.where(((alignment_df.iloc[:, 0] - RT_error) <= sample_df.iloc[row, 0]) &\
                         (sample_df.iloc[row, 0] <= (alignment_df.iloc[:,0] + RT_error))\
                         & ((alignment_df.iloc[:, 1] - mz_error) <= sample_df.iloc[row, 1])\
                         & (sample_df.iloc[row, 1] <= (alignment_df.iloc[:, 1] + mz_error)))
                if overlap[0].size > 0: #checks if tuple will have size
                    overlap_val = pd.DataFrame(overlap)
                    # sets intensity value into slot where realignment was identified
                    alignment_df.at[overlap_val.iloc[0,0], align_sample] = sample_df.iloc[row, 2]
                    # adds RT time to aligned point
                    alignment_df.at[overlap_val.iloc[0,0], 'Sum RT (min)'] += sample_df.iloc[row,0]
                    # adds m/z value to aligned point
                    alignment_df.at[overlap_val.iloc[0,0], 'Sum Precursor m/z'] += sample_df.iloc[row,1]
                else:
                    # appends as a new reference if nothing is found
                    alignment_df = alignment_df.append(sample_df.iloc[row, 0:2], ignore_index = True)
                    alignment_df.at[alignment_df.index[-1], align_sample] = sample_df.iloc[row, 2]
                    alignment_df.at[alignment_df.index[-1], 'Sum RT (min)'] = sample_df.iloc[row,0]
                    alignment_df.at[alignment_df.index[-1], 'Sum Precursor m/z'] = sample_df.iloc[row,1]
        else:
            alignment_df['RT (min)'] = standard_df.iloc[:,0]
            alignment_df['Precursor m/z'] = standard_df.iloc[:,1]
            alignment_df['Sum RT (min)'] = standard_df.iloc[:,0]
            alignment_df['Sum Precursor m/z'] = standard_df.iloc[:,1]
            alignment_df[standard_sample] = standard_df.iloc[:,2]
            mz_error = 0.00002*standard_df.iloc[:,1] # 20 ppm  
    alignment_df.rename(columns = {'RT (min)':'Average RT (min)'}, inplace = True)
    alignment_df.rename(columns = {'Precursor m/z':'Average m/z'}, inplace = True)
    # Replace all NaN elements with 0
    alignment_df = alignment_df.fillna(0)
    for rows in range(len(alignment_df)):
        # Calculating the averages after the iterations
        # requires count of nonzero count to calculate the mean properly
        count = np.count_nonzero(alignment_df.iloc[rows, 4:])
        if count > 0:
            alignment_df.at[rows, 'Average RT (min)'] = alignment_df.loc[rows, 'Sum RT (min)']/count
            alignment_df.at[rows, 'Average m/z'] = alignment_df.loc[rows, 'Sum Precursor m/z']/count
        else:
            pass
    # Final sort by m/z
    #alignment_df = alignment_df.sort_values(by='Average m/z')
    # converts file back to .txt
    alignment_df.to_csv(batch_name + '.txt', header=True, index=True)
    return alignment_df

In [302]:
df = realignment('/mnt/c/Users/Derek/Desktop/mass-suite/example_data/batch/', 'test batch')

100%|██████████| 2/2 [00:04<00:00,  2.38s/it]


In [305]:
df

,Average RT (min),Average m/z,Sum RT (min),Sum Precursor m/z,test1.txt,test2.txt
0,0.578250,179.03580,0.578250,179.0358,26772.347656,0.000000
1,0.582975,201.07110,1.165950,402.1422,9090.625000,8615.000000
2,0.582975,217.04935,1.165950,434.0987,17694.775391,19556.000000
3,0.588508,142.03450,1.177017,284.0690,17614.511719,17208.000000
4,0.588508,158.01210,1.177017,316.0242,64892.605469,82808.000000
...,...,...,...,...,...,...
1382,21.106630,201.00350,21.106630,201.0035,0.000000,61065.386719
1383,21.123230,354.94110,21.123230,354.9411,0.000000,20612.689453
1384,21.128770,390.91360,21.128770,390.9136,0.000000,10664.412109
1385,21.161970,181.98080,21.161970,181.9808,0.000000,29220.789062


In [292]:
df

,Average RT (min),Average m/z,Sum RT (min),Sum Precursor m/z,test1.txt,test2.txt,test3.txt
0,0.578250,179.035800,0.578250,179.0358,26772.347656,0.0,0.000000
1,0.582611,201.072600,1.747833,603.2178,9090.625000,8615.0,7226.000000
2,0.582611,217.049333,1.747833,651.1480,17694.775391,19556.0,20366.000000
3,0.588144,142.034533,1.764433,426.1036,17614.511719,17208.0,16375.000000
4,0.588144,158.012300,1.764433,474.0369,64892.605469,82808.0,63681.000000
...,...,...,...,...,...,...,...
1596,21.111830,390.913600,21.111830,390.9136,0.000000,0.0,15806.545898
1597,21.117370,201.006900,21.117370,201.0069,0.000000,0.0,44081.074219
1598,21.133970,181.979000,21.133970,181.9790,0.000000,0.0,29594.367188
1599,21.189300,157.976300,21.189300,157.9763,0.000000,0.0,82636.648438


In [289]:
np.count_nonzero(df.iloc[1600, 4:])

1

In [306]:
%time realignment('/mnt/c/Users/Derek/Desktop/mass-suite/example_data/0815/0815/', 'test batch 2')

 59%|█████▉    | 74/125 [53:23<36:47, 43.29s/it]   


KeyboardInterrupt: 

In [243]:
path = os.listdir(r'/mnt/c/Users/Derek/Desktop/mass-suite/example_data/batch/')[0]
path
ref_df = pd.read_csv(standard_sample, sep="\\t", engine='python')

'test1.txt'

In [161]:
df = pd.read_csv('/mnt/c/Users/Derek/Desktop/mass-suite/dev/test batch 2.txt', sep=",", engine='python')

In [169]:
np.count_nonzero(df.iloc[100000,3])

0

In [128]:
np.count_nonzero(df.iloc[:,3:-1])

243235